In [1]:
!nvidia-smi

Wed Jun 16 12:44:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader
import nltk
from rouge import Rouge

In [3]:
rouge = Rouge()

In [4]:
datasets = './datasets/'

In [5]:
model_loc = './table_qgen/'

In [6]:
tok = T5TokenizerFast.from_pretrained(model_loc)
model = T5ForConditionalGeneration.from_pretrained(model_loc).cuda()

In [7]:
test = pd.read_csv(f'{datasets}/tabqgen_test.csv')
test.drop(columns = ['Unnamed: 0'], inplace = True)

In [8]:
test_input_txt = test.subtable_text.values
test_target_txt = test.questions.values

In [9]:
test_input_txt[1], test_target_txt[1]

('<page_title> Alma Jodorowsky </page_title> <section_title> Filmography </section_title> <table> <cell> 2016 <col_header> Year </col_header> </cell> <cell> Kids in Love <col_header> Title </col_header> </cell> <cell> Evelyn <col_header> Role </col_header> </cell> </table>',
 'What role did Alma play in the movie Kids in Love?')

In [10]:
def encode_input(inp_txt):
    enc_inp_ids = []
    enc_inp_attn_mask = []

    for in_txt in tqdm_notebook(inp_txt):
        enc_input = tok.encode_plus(in_txt, 
                                      add_special_tokens=True,
                                      max_length = 128,
                                      truncation = True,
                                      return_tensors = 'pt',
                                      return_attention_mask=True,
                                      return_token_type_ids=False,
                                      padding = 'max_length')
        enc_inp_ids.append(enc_input.input_ids)
        enc_inp_attn_mask.append(enc_input.attention_mask)

    enc_inp_ids = torch.cat(enc_inp_ids, dim = 0)
    enc_attn_mask = torch.cat(enc_inp_attn_mask, dim = 0)
    
    return(enc_inp_ids, enc_attn_mask)

In [11]:
test_enc_inp_ids, test_enc_attn_mask = encode_input(test_input_txt)

  0%|          | 0/7700 [00:00<?, ?it/s]

In [12]:
outf = open(f'test_preds.txt', 'w')
for idx in tqdm_notebook(range(len(test_enc_inp_ids))):
    out = model.generate(input_ids=test_enc_inp_ids[idx].reshape(1,512).to('cuda'))
    pred_q = tok.batch_decode(out.detach().cpu().numpy(), skip_special_tokens=True)[0]
    outf.write(f'{pred_q}\n')
outf.close()

  0%|          | 0/7700 [00:00<?, ?it/s]